In [1]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

base_dir = Path.cwd()
parent_dir = base_dir.parent

In [2]:
# si_2018 = load_csv_from_raw('si_2018.csv')

# si_2018.iloc[:, 19:24].head()

In [3]:
si = merge_si()
ss = merge_ss()

Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported si_all.csv to /home/jovyan/shared/service-data/outputs/utils
Exported sid_list.csv to /home/jovyan/shared/service-data/outputs/utils
Exported dept.csv to /home/jovyan/shared/service-data/outputs/utils
Exported ss_all.csv to /home/jovyan/shared/service-data/outputs/utils


In [4]:
si = si.loc[si['service_scope_ext_or_ent']]

si_inscope = si.set_index(['fiscal_yr','service_id'])

ss_inscope = ss.set_index(['fiscal_yr', 'service_id']).merge(si_inscope['service_scope_ext_or_ent'], how='left', left_index=True, right_index=True)
ss_inscope = ss_inscope.loc[ss_inscope['service_scope_ext_or_ent'].notna()].reset_index()


In [5]:
ss = ss_inscope

In [9]:
# MAF Question 1: Existence of service standards
# As service standards are required under the Policy on Service and Digital, what is the percentage of services that have service standards?

# Select relevant columns from service inventory
maf1 = si.loc[:, ['fiscal_yr', 'service_id', 'department_en','department_fr', 'org_id']]

# Deduplicate service standards to prevent one-to-many expansion
ss_unique = ss[['fiscal_yr', 'service_id']].drop_duplicates()

# Determine whether each service has a standard by checking for existence in 'service standards'
# Merge with 'ss' to check if (fiscal_yr, service_id) exists
maf1 = maf1.merge(
    ss_unique,  # Use de-duplicated version to check
    on=['fiscal_yr', 'service_id'],  # Merge on fiscal year and service ID
    how='left',  # Keep all 'maf1' records, add matches from 'ss'
    indicator=True  # Adds a column "_merge" to show if a match was found
)

# Create boolean column: True if the service exists in 'ss', otherwise False
maf1['service_std_tf'] = maf1['_merge'] == 'both'

# Drop the '_merge' column (no longer needed)
maf1 = maf1.drop(columns=['_merge'])

# Group by department and fiscal year, counting services with and without standards
maf1 = maf1.groupby(['fiscal_yr', 'department_en', 'department_fr', 'org_id']).agg(
    service_with_std_count=('service_std_tf', 'sum'),  # Count services that have standards (True = 1)
    service_count_maf1=('service_id', 'count')  # Count all services
).reset_index()

maf1['maf1_score'] = (maf1['service_with_std_count']/maf1['service_count_maf1'])*100

In [7]:
# maf2 = ss.loc[:, ['fiscal_yr', 'service_standard_id', 'department_en','department_fr', 'org_id', 'target_met']].dropna()
    
# maf2_num = maf2[maf2['target_met']=='Y'].groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()
# maf2_denom = maf2.groupby(['fiscal_yr', 'department_en','department_fr', 'org_id'])['service_standard_id'].count().reset_index()

# maf2 = pd.merge(
#     maf2_num,
#     maf2_denom,
#     suffixes=['_met','_total'],
#     on=['fiscal_yr', 'department_en','department_fr', 'org_id'],
#     how='outer'
# )

# maf2['maf2_score'] = (maf2['service_standard_id_met']/maf2['service_standard_id_total'])*100

# maf2.loc[maf2['fiscal_yr'] == '2023-2024', ['service_standard_id_total']].sum()

In [8]:


# ifoi_en = pd.read_csv(parent_dir / "inputs" / "ifoi_en.csv")
# ifoi_fr = pd.read_csv(parent_dir / "inputs" / "ifoi_fr.csv")

# ifoi_en = ifoi_en.set_index(ifoi_en.columns[0], drop=True).add_suffix('_en')
# ifoi_fr = ifoi_fr.set_index(ifoi_fr.columns[0], drop=True).add_suffix('_fr')

# ifoi_en.set_index(ifoi_en.columns[0], drop=True)
# ifoi_fr.set_index(ifoi_fr.columns[0], drop=True)

# ifoi = pd.concat([ifoi_en, ifoi_fr], axis=1)

# # Extract column lists
# en_cols = ifoi_en.columns.tolist()
# fr_cols = ifoi_fr.columns.tolist()

# # Interleave them by index
# merged_cols = [col for pair in zip(en_cols, fr_cols) for col in pair]

# # Apply new column order
# ifoi = standardize_column_names(ifoi[merged_cols].reset_index())

# ifoi